In [ ]:
import signac
import panedr
import numpy as np
import unyt as u

In [ ]:
project = signac.get_project()

In [ ]:
pretty_names = {
    "cassandra": "Cassandra",
    "mcccs": "MCCCS-MN",
    "gomc": "GOMC",
    "gromacs": "GROMACS",
    "hoomd": "HOOMD-blue",
    "lammps-VU": "LAMMPS",
}
pretty_molecules = {
    "methaneUA": "TraPPE Methane",
    "pentaneUA": "TraPPE Pentane",
    "benzeneUA": "TraPPE Benzene",
    "ethanolAA": "OPLS-AA Ethanol",
    "waterSPCE": "SPC/E Water"}

In [ ]:
engines = ["lammps-VU",
           "hoomd",
           "gromacs",
           "mcccs",
           "gomc",
           "cassandra"]

molecules = ["methaneUA", "pentaneUA", "benzeneUA", "waterSPCE", "ethanolAA"]
properties = ['potential_energy',
        'tot_vdw_energy',
        'tail_energy',
        'tot_electrostatics',
        'short_range_electrostatics',
        'long_range_electrostatics',
        'tot_pair_energy',
        'bonds_energy',
        'angles_energy',
        'dihedrals_energy',
        'tot_bonded_energy',
        'intramolecular_energy',
        'intermolecular_energy',]

In [ ]:
spe_data = dict()
for molecule in molecules:
    spe_data[molecule] = dict()
    for engine in engines:
        spe_data[molecule][engine] = dict()
        for property in properties:
            spe_data[molecule][engine][property] = 0

# Parse LAMMPS SPE

In [ ]:
lmp_map = {"potential_energy": "potential_energy",
           "tot_vdw_energy": "tot_vdw_energy",
           "short_range_electrostatics": "tot_electrostatics",
           "tot_electrostatics": "tot_electrostatics",
           "pair_energy": "tot_pair_energy",
           "bonds_energy": "bonds_energy",
           "angles_energy": "angles_energy",
           "dihedrals_energy": "dihedrals_energy",
           "tail_energy": "tail_energy",
           "long_range_electrostatics": "long_range_electrostatics",}

In [ ]:
for job in project.find_jobs({"engine": "lammps-VU"}):
    data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

In [ ]:
# for job in project.find_jobs({"engine": "lammps-VU"}):
#     data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
#     for prop in data.dtype.names:
#         spe_data[job.sp.molecule][job.sp.engine][lmp_map[prop]] += data[prop]

# Parse HOOMD SPE

In [ ]:
properties = ['potential_energy',
        'tot_vdw_energy',
        'tail_energy',
        'tot_electrostatics',
        'short_range_electrostatics',
        'long_range_electrostatics',
        'tot_pair_energy',
        'bonds_energy',
        'angles_energy',
        'dihedrals_energy',
        'tot_bonded_energy',
        'intramolecular_energy',
        'intermolecular_energy',]

In [ ]:
hoomd_map = {"pair_LJ": ("tot_pair_energy", "tot_vdw_energy",),
             "pair_LJ_tail": "tail_energy",
             "pair_Ewald": ("short_range_electrostatics", "tot_electrostatics",),
             "pppm_Coulomb": ("long_range_electrostatics", "tot_electrostatics",),
             "special_pair_LJ": "tot_pair_energy",
             "special_pair_Coulomb": ("short_range_electrostatics", "tot_electrostatics"),
             "bond_Harmonic": "bonds_energy",
             "angle_Harmonic": "angles_energy",
             "dihedral_OPLS": "dihedrals_energy",
             "potential_energy": "potential_energy"}

In [ ]:
for job in project.find_jobs({"engine": "hoomd"}):
    data = np.genfromtxt(f"{job.ws}/log-spe-raw.txt", names=True, delimiter=" ")
    for prop in data.dtype.names:
        if isinstance(hoomd_map[prop], str):
            spe_data[job.sp.molecule][job.sp.engine][hoomd_map[prop]] += data[prop]
        else:
            for key in hoomd_map[prop]:
                spe_data[job.sp.molecule][job.sp.engine][key] += data[prop]


# Parse GROMACS SPE


In [ ]:
for job in project.find_jobs({"engine": "gromacs"}):
    data = np.genfromtxt(f"{job.ws}/log-spe-p3m.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

# Parse MCCCS SPE

In [ ]:
for job in project.find_jobs({"engine": "mcccs"}):
    data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

# Parse Cassandra SPE

In [ ]:
for job in project.find_jobs({"engine": "cassandra"}):
    data = np.genfromtxt(f"{job.ws}/log-spe.txt", names=True, delimiter=",")
    for prop in data.dtype.names:
        if prop in properties:
            spe_data[job.sp.molecule][job.sp.engine][prop] += data[prop]
        else:
            print(f"Not included {job.sp.molecule} {prop}")

# Create log-spe with full column from GOMC

In [ ]:
# get_e_titles = bool as there are 2 headers, and it will print both otherwise
# log_file_splitline_iter[1] == '0': means box 0 (typicially liquid),  1= box 1 (typically vapor)

In [ ]:
def K_to_kj_per_mol_conversion(value):
    conversion_factor = (1 * u.Kelvin).to_value("kJ/mol", equivalence="thermal") #conversion_K_to_kj_per_mol
    converted = value * conversion_factor
    return converted

In [ ]:
gomc_map = {
    "TOTAL": "potential_energy",
    "INTRA(B)": "intramolecular_energy",
    "INTRA(NB)": ("intramolecular_energy", "tot_vdw_energy", "tot_pair_energy"),
    "BOND(B)": "bonds_energy",
    "ANGLE(B)": "angles_energy",
    "DIHEDRAL(B)": "dihedrals_energy",
    "INTER(LJ)": ("intermolecular_energy", "tot_vdw_energy", "tot_pair_energy"),
    "LRC": ("tail_energy", "tot_vdw_energy"),
    "TOTAL_ELECT": "tot_electrostatics",
}

In [ ]:
for job in project.find_jobs({"engine": "gomc"}):
    with open(f"{job.ws}/out_production_run.dat") as f:
        data = f.readlines()
    for i, line in enumerate(data):
        if "INITIAL SIMULATION ENERGY" in line:
            for title, value in zip(data[i+2].split(), data[i+4].split()):
                 # spe_data[job.id][title] = value
                if title in gomc_map:
                    if isinstance(gomc_map[title], str):
                        spe_data[job.sp.molecule][job.sp.engine][gomc_map[title]] += K_to_kj_per_mol_conversion(float(value))
                    else:
                        for key in gomc_map[title]:
                            spe_data[job.sp.molecule][job.sp.engine][key] += K_to_kj_per_mol_conversion(float(value))
                else:
                    print(f"Not included {job.sp.molecule} {title}")

# Summary

In [ ]:
import pandas as pd

summarize_dfs = dict()
for molecule in spe_data:
    print(molecule)
    df = pd.DataFrame.from_dict(spe_data[molecule]).transpose()
    display(df)
    df.to_csv(f"csvs/{molecule}_spe.csv")

### Note for GOMC energy
- `INTRA (NB)` include short range electrostatic --> got included in `tot_vdw_energy`
- Short range electrostatic is not part of `tot_electrostatics` 

In [ ]:
spe_data["methaneUA"]["gromacs"]

In [ ]:
potential_energy_table = dict()

engines = ("lammps-VU", "hoomd", "gromacs",
                    "mcccs", "gomc", "cassandra")
molecules = ("methaneUA", "pentaneUA", "benzeneUA", "ethanolAA", "waterSPCE")

for engine in engines:
    potential_energy_table[pretty_names[engine]] = list()
    for molecule in molecules:
        potential_energy_table[pretty_names[engine]].append(
            spe_data[molecule][engine]["potential_energy"])

potential_energy_df = pd.DataFrame.from_dict(potential_energy_table,
                       orient="index",
                       columns=[pretty_molecules[molecule] for molecule in molecules])

potential_energy_df.to_csv("csvs/potential_energy.csv")


In [ ]:
electrostatics_table = dict()

engines = ("lammps-VU", "hoomd", "gromacs",
                    "mcccs", "gomc", "cassandra")
molecules = ("methaneUA", "pentaneUA", "benzeneUA", "ethanolAA", "waterSPCE")

for engine in engines:
    electrostatics_table[pretty_names[engine]] = list()
    for molecule in molecules:
        electrostatics_table[pretty_names[engine]].append(
            spe_data[molecule][engine]["tot_electrostatics"])

electrostatics_df = pd.DataFrame.from_dict(electrostatics_table,
                       orient="index",
                       columns=[pretty_molecules[molecule] for molecule in molecules])

electrostatics_df.to_csv("csvs/electrostatics_energy.csv")


In [ ]:
vdw_table = dict()

engines = ("lammps-VU", "hoomd", "gromacs",
                    "mcccs", "gomc", "cassandra")
molecules = ("methaneUA", "pentaneUA", "benzeneUA", "ethanolAA", "waterSPCE")

for engine in engines:
    vdw_table[pretty_names[engine]] = list()
    for molecule in molecules:
        vdw_table[pretty_names[engine]].append(
            spe_data[molecule][engine]["tot_vdw_energy"])

vdw_df = pd.DataFrame.from_dict(vdw_table,
                       orient="index",
                       columns=[pretty_molecules[molecule] for molecule in molecules])

vdw_df.to_csv("csvs/vdw_energy.csv")
